In [1]:
!pip install kaggle

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
%matplotlib inline

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.head()

label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0         0         0         0   
3       0  ...         0         0         0         0         0         0   
4       0  ...         0         0         0         0         0         0   

   pixel780  pixel781  pixel782  pixel783  
0         0         0         0         0  
1         0         0         0         0  
2         0         0         0         0  
3         0         0         0         0  
4         0         0         0         0  

[5 rows x 785 columns]

In [4]:
train.shape, test.shape

((42000, 785), (28000, 784))

In [5]:
train_y = train['label'].astype('float32')
train_x = train.drop(['label'],axis = 1).astype('int32')
test_x = test.astype('float32')
train_x.shape, train_y.shape, test_x.shape

((42000, 784), (42000,), (28000, 784))

In [6]:
train_x = train_x.values.reshape(-1,28,28,1)
train_x = train_x / 255.0
test_x = test_x.values.reshape(-1,28,28,1)
test_x = test_x / 255.0
train_x.shape, test_x.shape

((42000, 28, 28, 1), (28000, 28, 28, 1))

In [7]:
train_y = tf.keras.utils.to_categorical(train_y,10)
train_y.shape

(42000, 10)

In [8]:
train['label']

0        1
1        0
2        1
3        4
4        0
        ..
41995    0
41996    1
41997    7
41998    6
41999    9
Name: label, Length: 42000, dtype: int64

In [9]:
cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'Same'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [10]:
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('accuracy') > 0.999):
                print("\nReached 99.9% accuracy so cancelling training!")
                self.model.stop_training = True
    
callbacks = myCallback()

In [11]:
#Compiling and model training with batch size = 50, epochs = 20, and optimizer = adam
Optimizer = tf.keras.optimizers.Adam(
            learning_rate=0.0005, 
            beta_1=0.9, 
            beta_2=0.999, 
            epsilon=1e-07,
            name='Adam'
)
cnn.compile(optimizer=Optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
cnn.fit(train_x, train_y, batch_size = 50, epochs = 20, callbacks=[callbacks])

Epoch 1/20
840/840 [==============================] - 26s 31ms/step - loss: 0.2012 - accuracy: 0.9406
Epoch 2/20
840/840 [==============================] - 26s 31ms/step - loss: 0.0559 - accuracy: 0.9827
Epoch 3/20
840/840 [==============================] - 26s 31ms/step - loss: 0.0392 - accuracy: 0.9881
Epoch 4/20
840/840 [==============================] - 28s 33ms/step - loss: 0.0278 - accuracy: 0.9917
Epoch 5/20
840/840 [==============================] - 28s 33ms/step - loss: 0.0226 - accuracy: 0.9927
Epoch 6/20
840/840 [==============================] - 29s 34ms/step - loss: 0.0164 - accuracy: 0.9951
Epoch 7/20
840/840 [==============================] - 26s 30ms/step - loss: 0.0135 - accuracy: 0.9953
Epoch 8/20
840/840 [==============================] - 26s 31ms/step - loss: 0.0104 - accuracy: 0.9964
Epoch 9/20
840/840 [==============================] - 25s 30ms/step - loss: 0.0078 - accuracy: 0.9974
Epoch 10/20
840/840 [==============================] - 26s 30ms/step - loss: 0.006

In [12]:
predictions = cnn.predict(test_x)

# select the index with the maximum probability
predictions = np.argmax(predictions,axis = 1)

predictions = pd.Series(predictions,name="Label")

875/875 [==============================] - 5s 5ms/step


In [13]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),predictions],axis = 1)

submission.to_csv("submission.csv",index=False)